### Resources

In addition to the jupyter notebooks included in this github repository, there are a number of tutorials that may end up being useful to you as you progress through the research. 

Some useful links:

- Making images with Pynbody:

https://pynbody.readthedocs.io/v1-docs/tutorials/pictures.html

- Once we start using Tangos:

https://nbviewer.jupyter.org/github/pynbody/tangos/blob/master/docs/Data%20exploration%20with%20python.ipynb

# Measuring OVI 

In [ ]:
# Copy and paste the packages you usually load in here
# Don't forget to include your module for loading simulations

So, one of your key goals this summer is to calculate the amount of hot gas (traced by OVI) that is in the CGM of your galaxy. So you have a few important steps to do this calculation for each of your galaxies:
1. Define your CGM
2. Calculate the amount of OVI
3. Calculate the average column density of OVI, $N_{OVI}$ in the CGM
4. Plot the $N_{OVI}$ vs $M_{*}$ for every galaxy in your sample

## 1. Define the CGM

To define the CGM, you need to select all the particles that are *outside the disk of the galaxy* and *inside the virial radius*. Due to the way that the simulations were prepared for you, you don't have to worry about the outer radius, you ONLY have gas particles from inside the virial radius of your galaxies. But, you do need to remove the interstellar medium (i.e. the gas inside the disk and bulge of the galaxy). 

To do this selection, you can use the following steps:

1. Define an object called all_gas
 
```all_gas = hnumber.g```

2. Decide on a radius of your galaxy (let's start with 10 kpc for now); we are going to remove all the gas particles inside this radius

```disk_radius = 10```

3. From all_gas, select only the gas particles outside disk_radius

```all_gas['r'] > 10```

You can see that this is an array of False and True. This array tells us which particles are inside or outside of 10 kpc. We can use this array like a selection tool, aka an index, for the all_gas array so that it only selects the particles outside 10 kpc

```all_gas[all_gas['r'] > 10]```

4. Call this new array: CGM_gas

```CGM_gas = all_gas[all_gas['r'] > 10]```

5. Print the length of all_gas and CGM_gas and you'll see that you've removed a large number of particles.

6. Try plotting a SIDEON image of CGM_gas, and you should see a sphere of particles missing from the center of the halo.

In [ ]:
# Define an object called all_gas


In [ ]:
# Define the a radius of your galaxy


In [ ]:
# Select only the gas particles outside disk_radius


In [ ]:
# Define array CGM_gas


In [ ]:
# Print the length of all_gas and CGM_gas


In [ ]:
# Plot a SIDEON image of CGM_gas


## 2. Calculate the amount of OVI in the CGM

### Important note: Our simulations directly track oxygen and iron, 
but otherwise we assume that our galaxies follow solar abundances for other elements (Citation: Asplund 2009)

The equation to calculate the OVI in every particle looks like:
\begin{equation}
M_{OVI} = M_{gas} \times f_{oxygen\ mass} \times f_{OVI\ mass}
\end{equation}

where $M_{gas}$ is the mass of every gas particle, f_{oxygen\ mass} is the fraction of gas mass in oxygen, and f_{OVI\ mass} if the fraction of oxygen mass in OVI.

### OVI in an entire halo
You will first calculate the amount of OVI gas mass in the CGM of one entire galaxy in the cell below. Before that, I've broken down each of the steps in detail:

1. First, you will need *the total amount of gas mass* per particle

   - As you have previous seen, the array containing all the gas mass in the galaxy looks like this:

     ```hnumber.g['mass']```
  
2. Then you want to *calculate the amount of oxygen mass*
   
   - The array 'OxMassFrac' is the *oxygen mass fraction* of each gas particle. In other words, because each gas particle is really a cloud of gas, the 'OxMassFrac' quantity tells us "how much of the mass of each gas particle is made up of oxygen"

   - So, to calculate the amount of oxygen mass in each gas particle in the galaxy:

     ```hnumber.g['mass'] * hnumber.g['OxMassFrac']```

        *** Capitalization is important when calling column from the galaxy object/table
      
3. Finally, you need to use the function ```hdf5_ion_frac``` from Nicole to calculate the amount of OVI within the oxygen mass
   
   - Just like you used the oxygen mass fraction to calculate how much oxygen mass there was in each gas mass, you similarly need to use the function to get the OVI Mass Fraction to be able to calculate how much OVI mass is present in each gas particle.
  
   - To do get this fraction, you'll need two things
     
     1. The hm2012_hr.h5 table which you can download in the [2025 Google Drive](https://drive.google.com/drive/folders/1h7cVDVEkkeqG-oaOLKNW4jld2MWzdRer?usp=drive_link)
     2. The hdf5_Novi_R.py code from Nicole (should be included in the CASSI Research Github already)


    - You'll use Nicole's code using the following syntax:
  
      ``` from hdf5_Novi_R import hdf5_ion_frac ```

      ``` OVI_mass_frac = hdf5_ion_frac(sim=hg, element="O", ion=5) # OVI is 5 because it has 5 e- removed ```

    - Then, you can calculate:
  
      ```hnumber.g['mass'] * hnumber.g['OxMassFrac'] * OVI_mass_frac```

   - The resulting array is the *amount of OVI mass in every gas particle* in the galaxy.

4. Once you calculate the sum of the array, you'll have the total OVI mass for the galaxy!

   - Hint: Remember that ```np.sum(name_of_array)``` can add it all up for you 

In [ ]:
# I know it looks like a lot of steps, but remember that creating a new object for everything can make things easier to understand
# For example, to calculate the total mass of the OVI in the entire galaxy (using h19 for an example)
import pynbody
from hdf5_Novi_R import hdf5_ion_frac

# Load in your galaxy as usual using your module
h19 = pynbody.load('../R25_gsonly/R25_h18_gsonly')
h19.physical_units()

pynbody.analysis.angmom.faceon(h19.g)

# Define just the gas particles
h19_gas = h19.g

# You'll need three things: gas mass, oxygen mass fraction, OVI mass fraction
h19_gas_mass = h19_gas['mass']

h19_gas_oxygen_mass_fraction = h19_gas['OxMassFrac']

h19_gas_OVI_mass_fraction = hdf5_ion_frac(sim=h19, element="O", ion=5) # this function needs to full sim to calculate the OVI gas

# Then you can calculate the amount of mass in OVI gas using the equation above
h19_gas_OVI_mass = h19_gas_mass * h19_gas_oxygen_mass_fraction * h19_gas_OVI_mass_fraction

# Sum the h19_gas_OVI_mass to get the total amount of OVI gas in the galaxy

### Now, you are going to calculate the amount of all OVI gas mass $\underline{in\ the\ CGM}$

Use the instructions and the example code above to redo this calculation for the gas particles only in the CGM 
- For this calculation, define the CGM at 15 kpc as you previously have done

#### Important instruction from Nicole: As you are writing this code for the CGM, I want you to use one cell per line of code and use comments like I did above to write a short description of what each line in the code is doing. For example:

In [ ]:
# First, I need to load in additional functions I might need
import numpy as np

And so on.

### OVI in the CGM

In [ ]:
#### Finally, make a plot of the total OVI gas mass vs stellar mass

## 3. Calculate the average column density in the CGM

Now that you have calculated the values for the OVI mass in each gas particle and the OVI mass fraction, to be able to compare to observations, we need to calculate the column density of OVI, $N_{OVI}$.

If you look at the plot from the Dutta paper (it's in the intro keynote presentation), you'll see that Figure 6 is a plot of N(OVI) vs stellar mass. To make this, we need to combine the information from your previous calculation and the profile function you learned about in notebook 3.

In [ ]:
# You can add a row to your halo object/table the following way:
h19_gas['OVIMassFraction'] = h19_gas_OVI_mass_fraction

In [ ]:
# Then when you run the profile function, you now have a column that includes the OVI Mass Fraction you calculated
h19_gas_profile = pynbody.analysis.profile.Profile(h19_gas, vmin =.01, max=100)

# Calculate Novi for our plot

## To calculate the column density of OVI (which is the value that we need for the plot), we use an equation that compresses our data along one axis:

The equation looks like:

\begin{equation}
N_{OVI} = log_{10}(\frac{M_{gas} \times f_{oxygen\ mass} \times f_{OVI\ mass}}{16 * m_p} \times \frac{1}{bin\ size})
\end{equation}

where: $M_{gas}$ is the gas mass profile in units of grams, $f_{oxygen\ mass}$ is the oxygen mass fraction profile, $f_{OVI\ mass\ fraction}$ is the OVI mass fraction profile, $m_p$ is the mass of a proton in grams, and the bin size is the size of the bins used to calculate the profile.

In code, this equation will look something like this:

In [ ]:
Novi_profile = np.log10((h19_gas_profile['mass'].in_units('g')*h19_gas_profile['OxMassFrac']*h19_gas_profile['OVI']//(16*m_p))/h19_gas_profile._binsize.in_units('cm**2'))

#### Below, plot the Novi_profile vs profile['rbins'] just like you did in notebook 3

Don't forget you'll need to import matplotlib if you didn't already!

In [ ]:
# Plot here!

Nice plot! Now, we just have one last step in this phase. 

Since you are going to be plotting one value for each galaxy CGM instead of the profile, you'll need to take the average of the OVI profile. Google ```numpy average function``` to figure out how to take the average of the ```Novi_profile``` array.

In [ ]:
# Take the average here

## 4. Plot $N_{OVI}$ vs M_{*}

Calculate M_star for your galaxy as you've previously done:

```hnumber.s['mass].sum()``` 

for comparison to observations, you need to multiple this number by 0.6. 

***Ask Nicole about this when you get to this point.

In [ ]:
# Calculate Mstar*0.6

In [ ]:
# Using plt.scatter, plot the average NOVI of the galaxy CGM vs the galaxy's stellar mass
# Then look at Dutta+2025 Figure 6 and see if point looks like it matches the observations